# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-24 11:23:50] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-24 11:23:50] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-24 11:23:50] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-24 11:23:52] INFO server_args.py:1769: Attention backend not specified. Use fa3 backend by default.


[2026-01-24 11:23:52] INFO server_args.py:2658: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=128 avail_mem=76.27 GB):   0%|          | 0/20 [00:00<?, ?it/s]

Capturing batches (bs=80 avail_mem=76.14 GB):  20%|██        | 4/20 [00:00<00:01,  9.06it/s]

Capturing batches (bs=32 avail_mem=72.60 GB):  55%|█████▌    | 11/20 [00:00<00:00, 19.27it/s]

Capturing batches (bs=2 avail_mem=72.57 GB):  85%|████████▌ | 17/20 [00:01<00:00, 21.88it/s]

Capturing batches (bs=1 avail_mem=72.57 GB): 100%|██████████| 20/20 [00:01<00:00, 18.39it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Jun. I am 15 years old and I have been learning to code for 2 years. I am learning it for fun. I would like to learn how to code other than C. My first question is: How can I learn to code other than C?

Please also consider my experience with the following programming languages:

- Java (x64)
- Python (x64)
- C++ (x64)
- Rust (x64)
- Assembly (x64)
- C (x32)
- Assembly (x32)
- C (x64)
- C (x8
Prompt: The president of the United States is
Generated text:  attempting to manage a political campaign. The campaign will take place over a period of 50 days. Each day, the president plans to hold speeches to educate the public about the issues they are addressing. The president wants to ensure that the speeches are effective in reaching different demographics of the population. 

The president decides to analyze the data from previous campaigns to identify which type of speech (e.g., in-person, digital, face-to-face) is most effective in reaching a de

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [job title] at [company name], and I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [job title] at [company name], and I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [job title] at [company name], and I'm excited to meet you and learn more about you. What

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris. 

This statement is factually correct and provides a clear and concise overview of the capital city's location and significance in French society and politics. It is a widely recognized and well-known fact about Paris, making it an important point of reference for those interested in French culture, history, and politics. 

However, it is important to note that this statement may not be entirely accurate or complete, as there are many other cities in France that are also considered capitals, such as Lyon, Marseille, and Toulouse. Additionally, the term "capital" can have different meanings depending on the context, so it is important to be aware of

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by several key trends:

1. Increased integration with human intelligence: As AI becomes more advanced, it is likely to become more integrated with human intelligence, allowing it to learn and adapt in ways that are difficult for humans to do. This could lead to more sophisticated and personalized AI systems that can better understand and respond to human emotions and behaviors.

2. Greater reliance on data: AI systems will become more data-driven, with the ability to learn from vast amounts of data to improve their performance. This could lead to more efficient and effective AI systems that can handle a wide range of tasks with greater accuracy and speed.

3.



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Your Name], and I'm a [Your Job or Role]! 

I am passionate about [Your Passion Statement], and I'm always looking to improve my skills and knowledge. In my free time, I enjoy [Your Interests or Passion], like reading, coding, or hiking.

What's your favorite hobby, and why is it important to you? As a [Your Profession], I find [Your Hobby] to be incredibly rewarding and fulfilling. It allows me to disconnect from daily stresses and focus on what truly matters. I believe it makes me more resilient, creative, and optimistic. 

In summary, my goal is to

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris.

Paris is the capital and largest city of France. It is located on the Seine River and is the third largest metropolitan area in the world, with over 2. 5 million inhabitants. The city is home to many famou

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

],

 and

...

 including

 faster

 and

 more

 accurate

In [6]:
llm.shutdown()